In [7]:
# 进程与线程初识
import multiprocessing as mp
import threading as td

def job(a,d):
    print('aaa')
    
t1 = td.Thread(target=job, args=(1,2))
p1 = mp.Process(target=job, args=(1,2,))

if __name__ == '__main__':
    t1.start()
    p1.start()
    t1.join()
    p1.join()

aaa
aaa



In [15]:
# 输出结果存放至Queue
import multiprocessing as mp

def job(q):
    res =0
    for i in range(1000000):
        res += i + i ** 2 + i ** 3
    q.put(res)
    
if __name__ == '__main__':
    q = mp.Queue()
    p1 = mp.Process(target=job, args=(q,))
    p2 = mp.Process(target=job, args=(q,))
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    res1=q.get()
    res2=q.get()
    print('multicore:',res1+res2)

multicore: 499999666667166666000000


In [17]:
# 效率对比
import multiprocessing as mp
import threading as td
import time

# 多核/多进程
def multicore():
    q = mp.Queue()
    p1 = mp.Process(target=job, args=(q,))
    p2 = mp.Process(target=job, args=(q,))
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    res1=q.get()
    res2=q.get()
    print('multicore:',res1+res2)

# 多线程
def multithread():
    q = mp.Queue()
    t1 = td.Thread(target=job, args=(q,))
    t2 = td.Thread(target=job, args=(q,))
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    res1 = q.get()
    res2 = q.get()
    print('multithread:', res1 + res2)

def normal():
    res = 0
    for _ in range(2):
        for i in range(1000000):
            res += i + i ** 2 + i ** 3
    print('normal:',res)
    
if __name__ == '__main__':
     st = time.time()
     normal()
     st1 = time.time()
     print('normal time:', st1 - st)
     multithread()
     st2 = time.time()
     print('multithread time:', st2 - st1)
     multicore()
     st3 = time.time()
     print('multicore time:', st3 - st2)

normal: 499999666667166666000000
normal time: 1.2207930088043213
multithread: 499999666667166666000000
multithread time: 1.3046579360961914
multicore: 499999666667166666000000
multicore time: 0.6932880878448486


In [18]:
# 进程池
import multiprocessing as mp

def job(x):
    return x*x

def multicore():
    '''
    Pool默认调用是CPU的核数,传入processes可自定义CPU核数
    map()放入迭代参数,返回多个结果
    apply_async()只能放入一组参数,并返回一个结果,如果想得到map()的效果需要通过迭代
    '''
    pool = mp.Pool(processes=2)
    res = pool.map(job, range(10))
    print(res)
    '''
    apply_async()只能传递一个值，它只会放入一个核进行运算，传入的值因为必须是可迭代的，
    所以在传入值后需要加逗号，同时需要用get()方法获取返回值。
    '''
    res = pool.apply_async(job, (2,))
    multi_res = [pool.apply_async(job, (i,)) for i in range(10)]
    print(res.get()) # 获取单个结果
    print([res.get() for res in multi_res]) # 获取多个结果

if __name__ == '__main__':
    multicore()

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
4
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


In [19]:
# 共享内存
import multiprocessing as mp

'''
使用Value数据存储在一个共享的内存表中
d表示一个双精浮点类型，i表示一个带符号的整型
'''
value1 = mp.Value('i', 0)
value2 = mp.Value('d', 3.14)

'''
Array类，可以和共享内存交互，来实现在进程之间共享数据。
这里的Array和numpy中的不同，它只能是一维的，不能是多维的。
同样和Value 一样，需要定义数据形式，否则会报错。
'''
array = mp.Array('i', [1,2,3,4])

In [20]:
# 进程锁
# 不同进程争夺资源
import multiprocessing as mp
import time
def job(v, num):
    for _ in range(5):
        time.sleep(0.1)
        v.value += num
        print(v.value,end="\n")

def multicore():
    v = mp.Value('i',0) # 定义共享变量
    p1 = mp.Process(target=job, args=(v,1))
    p2 = mp.Process(target=job, args=(v,3)) # 设定不同的number看如何抢夺内存
    p1.start()
    p2.start()
    p1.join()
    p2.join()

if __name__ == '__main__':
    multicore()

1
4
5
8
9
12
13
16
17
20


### 通过锁机制解决争夺资源问题

In [21]:
import multiprocessing as mp
import time
def job(v, num, l):
    l.acquire() # 锁住
    for _ in range(5):
        time.sleep(0.1)
        v.value += num
        print(v.value,end="\n")
    l.release() # 释放

def multicore():
    l = mp.Lock() # 定义一个进程锁
    v = mp.Value('i',0) # 定义共享变量
    p1 = mp.Process(target=job, args=(v,1,l)) # 需要将lock传入
    p2 = mp.Process(target=job, args=(v,3,l)) # 设定不同的number看如何抢夺内存
    p1.start()
    p2.start()
    p1.join()
    p2.join()

if __name__ == '__main__':
    multicore()

1
2
3
4
5
8
11
14
17
20
